In [187]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas import Series
#from mpl_toolkits.basemap import Basemap
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [188]:
#Reading in data and looking at the number of rows and columns using shape 
aapl_newz = pd.read_csv('AAPL_Newz_Data.csv')
aapl_newz.shape

(12974, 4)

In [189]:
aapl_price = pd.read_excel('AAPL_Openclose.xlsx')
aapl_price.shape

(10160, 7)

In [190]:
aapl_price.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100453,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095213,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088224,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090408,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093029,73449600


In [191]:
#removing data less than 1st Jan 2015
aapl_price = aapl_price.drop(aapl_price.index[aapl_price['Date'] <= "2015-01-01"])
aapl_price.head()

,Date,Open,High,Low,Close,Adj Close,Volume
8589,2015-01-02,27.847500,27.860001,26.837500,27.332500,24.745998,212818400
8590,2015-01-05,27.072500,27.162500,26.352501,26.562500,24.048864,257142000
8591,2015-01-06,26.635000,26.857500,26.157499,26.565001,24.051126,263188400
8592,2015-01-07,26.799999,27.049999,26.674999,26.937500,24.388374,160423600
8593,2015-01-08,27.307501,28.037500,27.174999,27.972500,25.325430,237458000


In [192]:
aapl_price.shape

(1571, 7)

In [193]:
def diff_nxt_row(df, col, diff_col):
    ls = df[col].tolist()
    n = len(ls)
    diff_ls = []
    i = -1
    for row in range(0, n):
        i += 1
        if i < (n-1):
            diff_ls.append(ls[i+1] - ls[i])
        else:
            diff_ls.append(None)
    df[diff_col] = diff_ls     

In [194]:
diff_nxt_row(aapl_price, "Open", "fut_change_stock_price")
aapl_price = aapl_price.dropna()

In [195]:
aapl_price["Pos/Neg_fut_Change"] = aapl_price["fut_change_stock_price"].apply(lambda x: 1 if x>0 else 0)

In [196]:
#To check if data has been loaded properly
aapl_newz.head()

,Date,Newz_Headline,Newz_Brief,Source
0,NaN,Apple Stays on Top for Phone Sales in China,Apple secured the top spot for phone sales in ...,The
1,hours 13 ago,"S&P 500, Dow Close Lower, But Media Stocks Sup...",By Yasin Ebrahim Investing.com – The S&P 500 a...,Investing.com
2,NaN,Apple’s new fix-it policy is a drop in the buc...,CNBC spoke with iFixit CEO Kyle Wiens before A...,CNBC
3,hours 21 ago,Hong Kong court denies bail to former Stand Ne...,By Clare Jim and Sara Cheng HONG KONG (Reuters...,Reuters
4,NaN,"ProShares Metaverse ETF To Track Meta, Apple, ...","In late November 2021, two Canadian metaverse ...",CryptoVibes


In [197]:
#removing null values from Date columns & changing "hours ago" to date
aapl_newz = aapl_newz.drop(aapl_newz.index[aapl_newz['Date'].isnull()])
aapl_newz = aapl_newz.replace(["hours 13 ago","hours 21 ago"],"31-Dec-21")

In [198]:
aapl_newz['Date'] = pd.to_datetime(aapl_newz['Date'])
aapl_newz.dtypes

Date             datetime64[ns]
Newz_Headline            object
Newz_Brief               object
Source                   object
dtype: object

In [199]:
aapl_newz.head()

,Date,Newz_Headline,Newz_Brief,Source
1,2021-12-31,"S&P 500, Dow Close Lower, But Media Stocks Sup...",By Yasin Ebrahim Investing.com – The S&P 500 a...,Investing.com
3,2021-12-31,Hong Kong court denies bail to former Stand Ne...,By Clare Jim and Sara Cheng HONG KONG (Reuters...,Reuters
7,2021-12-30,2021: The Year in Review,"By Peter Nurse, Liz Moyer, Sam Boughedda and Y...",Investing.com
8,2021-12-30,"Women force change at Indian iPhone plant, sic...","By Sudarshan Varadhan, A. Ananthalakshmi and A...",Reuters
10,2021-12-30,U.S. calls on China and Hong Kong to release S...,WASHINGTON (Reuters) - U.S. Secretary of State...,Reuters


In [200]:
aapl_combined = aapl_price.merge(aapl_newz, how = 'left', left_on = 'Date', right_on = 'Date')
aapl_combined.shape

(10481, 12)

In [201]:
aapl_combined.isnull().sum()

Date                       0
Open                       0
High                       0
Low                        0
Close                      0
Adj Close                  0
Volume                     0
fut_change_stock_price     0
Pos/Neg_fut_Change         0
Newz_Headline             67
Newz_Brief                67
Source                    67
dtype: int64

In [202]:
aapl_combined = aapl_combined.dropna()

In [203]:
print(aapl_combined.isnull().sum())
print(aapl_combined.shape)

Date                      0
Open                      0
High                      0
Low                       0
Close                     0
Adj Close                 0
Volume                    0
fut_change_stock_price    0
Pos/Neg_fut_Change        0
Newz_Headline             0
Newz_Brief                0
Source                    0
dtype: int64
(10414, 12)


In [204]:
aapl_combined.to_csv("AAPL_Combined.csv", index = False)

In [ ]:
#conda install gensim
import gensim
from gensim import corpora

In [ ]:
#Preparing Document-Term Matrix

# Creating the term dictionary of our corpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

"""Run LDA model"""

#Running LDA Model
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Training LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)

"""Print 10 topics"""

print("Topic 1")
print(ldamodel.print_topics(num_topics=10, num_words=50)[0])
print("Topic 2")
print(ldamodel.print_topics(num_topics=10, num_words=50)[1])
print("Topic 3")
print(ldamodel.print_topics(num_topics=10, num_words=50)[2])
print("Topic 4")
print(ldamodel.print_topics(num_topics=10, num_words=50)[3])
print("Topic 5")
print(ldamodel.print_topics(num_topics=10, num_words=50)[4])
print("Topic 6")
print(ldamodel.print_topics(num_topics=10, num_words=50)[5])
print("Topic 7")
print(ldamodel.print_topics(num_topics=10, num_words=50)[6])
print("Topic 8")
print(ldamodel.print_topics(num_topics=10, num_words=50)[7])
print("Topic 9")
print(ldamodel.print_topics(num_topics=10, num_words=50)[8])
print("Topic 10")
print(ldamodel.print_topics(num_topics=10, num_words=50)[9])